## Data Processes: Second Assignment (first part)

### Grupo:
Víctor Morcuende Castell, 47315589N

Guillermo Nájera Lavid, 70845359T

Javier Rocamora García, 20081979N

Antonio Ruiz García, 06601574E

In [61]:
import pandas as pd
from sklearn import preprocessing
from collections import defaultdict
from sklearn.impute import SimpleImputer
import math

In [62]:
# Auxiliary functions

def diagnoseDate_to_ageDiagnosed(birthDate, diagnoseDate):
    age = pd.to_datetime(diagnoseDate).year-pd.to_datetime(birthDate).year
    return age

def deathDate_to_age(birthDate, deathDate):
    deathAge = pd.to_datetime(deathDate).year-pd.to_datetime(birthDate).year
    if math.isnan(deathAge):
        deathAge = 0
    return deathAge
def deathDate_to_survivalTime(diagnosisDate, deathDate):
    survivalTime = pd.to_datetime(deathDate).year-pd.to_datetime(diagnosisDate).year
    if math.isnan(survivalTime) or survivalTime < 0:
        survivalTime = -1
    return survivalTime

In [63]:
df = pd.read_excel("breast_cancer_data.xlsx", index_col='ehr')

# Deleting unused column and duplicated data
df.pop('Unnamed: 0')
df = df.drop_duplicates(keep='first')

# Duplicating the DataFrame in order to obtain the numerical variables
df_num = pd.DataFrame(data=df, columns=df.columns, index=df.index)
df_num.pop('side')
df_num.pop('neoadjuvant')
df_num.pop('grade')
df_num.pop('invasive')
df_num.pop('er_positive')
df_num.pop('pr_positive')
df_num.pop('her2_positive')
df_num.pop('hist_type')

# Dividing the DataFrame into categorical and numerical variables
num_cols = df_num.columns.tolist()
df_cat = df.drop(num_cols, axis=1)

We delete the NULL values of the categorical variables by using the Simple Imputer

In [64]:
df_cat.side = df_cat.side.apply(lambda x: 'unknown' if (x != 'left' and x != 'right') else x)
df_cat.invasive = df_cat.invasive.apply(lambda x: 0 if x != 1 else x)

# Imputation of nulls in categorical columns using Simple Imputer
imp_cat = SimpleImputer(strategy='most_frequent')
columns = df_cat.columns
index = df_cat.index
df_cat = pd.DataFrame(imp_cat.fit_transform(df_cat), columns=columns, index=index)

# Transforming categorical values into numerical variables
df_cat.neoadjuvant = df_cat.neoadjuvant.apply(lambda x: 0.0 if x == 'no' else 1.0)

Now we are about to subsitute the categorical labels that are represented by strings with numerical values, in order to avoid working with Strings.

In [65]:
# Taking some variables out as they are already converted into numerical values
df_aux = pd.DataFrame(data=df_cat, columns=df_cat.columns, index=df_cat.index)
df_cat.pop('neoadjuvant')
df_cat.pop('invasive')
df_cat.pop('er_positive')
df_cat.pop('pr_positive')
df_cat.pop('her2_positive')
num_cols = df_cat.columns.tolist()
df_aux = df_aux.drop(num_cols, axis=1)

# Using OneHotEncoder
ohe = preprocessing.OneHotEncoder(sparse=False)
df_cat_ohe = pd.DataFrame(ohe.fit_transform(df_cat), 
                          columns=ohe.get_feature_names_out(df_cat.columns.tolist()),
                          index=df_cat.index)

# Merge both DataFrames (df_cat_ohe and df_aux)
df_cat_def = df_cat_ohe.join(other=df_aux)

We change the value of the birth_date column to the age of each patient by applying the above declared function to the dataframe, this way we will be able to know the age of each patient at the time they were diagnosed

In [66]:
# Age at which the patient was diagnosed
ageDiagnosed = pd.Series(df_num.apply(lambda x: deathDate_to_age(x.birth_date, x.diagnosis_date), axis=1), name='age_diagnosed')

# Age at which the patient died, if so
ageDead = pd.Series(df_num.apply(lambda x: deathDate_to_age(x.birth_date, x.death_date), axis=1), name='age_dead')

# Time of survival since diagnosis, 100 in case of full recovery
survivalTime = pd.Series(df_num.apply(lambda x: deathDate_to_survivalTime(x.diagnosis_date, x.death_date), axis=1), name='survival_time')

# Changing variables
df_num.pop('birth_date')
df_num.pop('diagnosis_date')
df_num.pop('death_date')
df_num = pd.merge(left=df_num, right=ageDiagnosed, on='ehr')
df_num = pd.merge(left=df_num, right=ageDead, on='ehr')
df_num = pd.merge(left=df_num, right=survivalTime, on='ehr')

We delete the NULL values of the numerical variables by using the Simple Imputer

In [67]:
# Imputation of nulls in categorical columns using Simple Imputer
imp_num = SimpleImputer(strategy='mean')
columns = df_num.columns
index = df_num.index
df_num_def = pd.DataFrame(imp_num.fit_transform(df_num), columns=columns, index=index)
print(df_num_def)

           ki67  recurrence_year  menarche_age  menopause_age  pregnancy  \
ehr                                                                        
6849  19.582781      2016.307692     17.000000      51.000000    2.00000   
268   19.582781      2016.307692     12.000000      48.714286    2.00000   
1458   0.000000      2016.307692     11.000000      48.714286    2.00000   
2013  17.000000      2016.307692     12.829268      48.714286    1.81746   
1350  44.000000      2016.307692     14.000000      48.714286    3.00000   
...         ...              ...           ...            ...        ...   
191   10.000000      2016.307692     13.000000      53.000000    2.00000   
6482   1.000000      2016.307692     11.000000      53.000000    1.81746   
2564  20.000000      2016.307692     11.000000      48.714286    0.00000   
2730  20.000000      2016.307692     12.829268      48.000000    0.00000   
2376  10.000000      2016.307692     14.000000      48.714286    2.00000   

         ab

Now that all variables are numerical and do not have missing values, we can merge the categorical and numerical variables

In [68]:
df_preprocessed = pd.merge(left=df_cat_def, right=df_num_def, on='ehr')
print(df_preprocessed)

      side_left  side_right  side_unknown  grade_1.0  grade_2.0  grade_3.0  \
ehr                                                                          
6849        0.0         0.0           1.0        1.0        0.0        0.0   
268         0.0         0.0           1.0        0.0        1.0        0.0   
1458        0.0         0.0           1.0        1.0        0.0        0.0   
2013        0.0         0.0           1.0        0.0        0.0        1.0   
1350        0.0         0.0           1.0        0.0        1.0        0.0   
...         ...         ...           ...        ...        ...        ...   
191         0.0         0.0           1.0        0.0        1.0        0.0   
6482        0.0         0.0           1.0        0.0        1.0        0.0   
2564        0.0         0.0           1.0        0.0        0.0        1.0   
2730        0.0         0.0           1.0        0.0        1.0        0.0   
2376        0.0         0.0           1.0        1.0        0.0 

Now we are going to eliminate the labels that do not give relevant information but would increase the computational cost of our data treatments.
e.g: Columns with too many different values, such as descriptions or names.